# Python DevOps Session 3: Networking & APIs

## **Topic 5: Networking & APIs for DevOps**

### **Learning Objectives:**
1. Master HTTP requests with the `requests` library
2. Handle errors, retries, and timeouts professionally
3. Interact with REST APIs (GitHub API examples)
4. Authenticate with APIs using tokens and OAuth
5. Implement SSH automation with `paramiko`
6. Build production-ready API clients
7. Create remote server management tools

---

### **What You'll Build:**
- REST API client with error handling
- GitHub repository automation tools
- Retry mechanisms with exponential backoff
- SSH-based server management scripts
- Automated deployment tools
- API rate limit handling
- Secure credential management

## **Section 1: Introduction to HTTP Requests**

### **Theory: HTTP and REST APIs**

HTTP (Hypertext Transfer Protocol) is the foundation of web communication and REST APIs.

**Key HTTP Methods:**
- **GET**: Retrieve data (read-only, idempotent)
- **POST**: Create new resources (not idempotent)
- **PUT**: Update/replace entire resource (idempotent)
- **PATCH**: Partial update of resource
- **DELETE**: Remove resource (idempotent)

**HTTP Status Codes:**
- **2xx Success**: 200 OK, 201 Created, 204 No Content
- **3xx Redirection**: 301 Moved Permanently, 302 Found
- **4xx Client Error**: 400 Bad Request, 401 Unauthorized, 403 Forbidden, 404 Not Found, 429 Too Many Requests
- **5xx Server Error**: 500 Internal Server Error, 502 Bad Gateway, 503 Service Unavailable

**REST API Principles:**
- Stateless communication
- Resource-based URLs
- Standard HTTP methods
- JSON/XML data formats
- Proper status codes

### **Essential HTTP Libraries:**
- **`requests`**: Simple, elegant HTTP library (recommended)
- **`urllib3`**: Lower-level HTTP client
- **`httpx`**: Modern async HTTP client

### **Example 1.1: Basic HTTP Requests**

In [ ]:
# Install required packages
%pip install requests paramiko

In [1]:
import requests
import json
from typing import Dict, Any, Optional
import time

# Example 1: Simple GET request
print("Example 1: Simple GET Request\n")

response = requests.get('https://httpbin.org/get')

print(f"Status Code: {response.status_code}")
print(f"Status: {'✓ Success' if response.ok else '✗ Failed'}")
print(f"Content Type: {response.headers['Content-Type']}")
print(f"Response Time: {response.elapsed.total_seconds():.2f}s")

# Parse JSON response
data = response.json()
print(f"\nResponse Data (origin): {data['origin']}")

# Example 2: GET with query parameters
print("\n" + "="*60)
print("Example 2: GET with Query Parameters\n")

params = {
    'name': 'DevOps',
    'course': 'Python',
    'level': 'advanced'
}

response = requests.get('https://httpbin.org/get', params=params)
print(f"Request URL: {response.url}")
print(f"Query Params Sent: {response.json()['args']}")

# Example 3: POST request with JSON data
print("\n" + "="*60)
print("📡 Example 3: POST Request with JSON\n")

payload = {
    'username': 'devops_user',
    'action': 'deploy',
    'environment': 'production',
    'timestamp': time.time()
}

response = requests.post(
    'https://httpbin.org/post',
    json=payload  # Automatically sets Content-Type: application/json
)

print(f"Status Code: {response.status_code}")
received_data = response.json()['json']
print(f"Data Sent: {json.dumps(received_data, indent=2)}")

# Example 4: Custom headers
print("\n" + "="*60)
print("📡 Example 4: Request with Custom Headers\n")

headers = {
    'User-Agent': 'DevOps-Bot/1.0',
    'Accept': 'application/json',
    'X-Custom-Header': 'DevOps-Session-3'
}

response = requests.get('https://httpbin.org/headers', headers=headers)
print(f"Headers Sent:")
for key, value in response.json()['headers'].items():
    if key.startswith('X-') or key in ['User-Agent', 'Accept']:
        print(f"  {key}: {value}")

Example 1: Simple GET Request

Status Code: 200
Status: ✓ Success
Content Type: application/json
Response Time: 14.61s

Response Data (origin): 109.229.5.10

Example 2: GET with Query Parameters

Status Code: 200
Status: ✓ Success
Content Type: application/json
Response Time: 14.61s

Response Data (origin): 109.229.5.10

Example 2: GET with Query Parameters

Request URL: https://httpbin.org/get?name=DevOps&course=Python&level=advanced
Query Params Sent: {'course': 'Python', 'level': 'advanced', 'name': 'DevOps'}

📡 Example 3: POST Request with JSON

Request URL: https://httpbin.org/get?name=DevOps&course=Python&level=advanced
Query Params Sent: {'course': 'Python', 'level': 'advanced', 'name': 'DevOps'}

📡 Example 3: POST Request with JSON

Status Code: 200
Data Sent: {
  "action": "deploy",
  "environment": "production",
  "timestamp": 1765648971.7268963,
  "username": "devops_user"
}

📡 Example 4: Request with Custom Headers

Status Code: 200
Data Sent: {
  "action": "deploy",
  "env

### **Explanation:**

**Basic Request Methods:**
- **`requests.get(url)`**: Send HTTP GET request
- **`requests.post(url, json=data)`**: Send POST request with JSON body
- **`response.status_code`**: HTTP status code (200, 404, etc.)
- **`response.ok`**: True if status code < 400
- **`response.json()`**: Parse JSON response body

**Query Parameters:**
- **`params=dict`**: Automatically URL-encodes and appends to URL
- Converts `{'key': 'value'}` to `?key=value`
- Handles special characters and spaces automatically

**Request Headers:**
- **`headers=dict`**: Send custom HTTP headers
- **`User-Agent`**: Identifies your client
- **`Accept`**: Specifies expected response format
- **Custom headers**: Use `X-` prefix for custom headers

**JSON Handling:**
- **`json=data`** parameter: Automatically serializes dict to JSON
- Sets `Content-Type: application/json` header
- **`response.json()`**: Deserializes JSON response to dict

**DevOps Use Case:** API integration, health checks, webhook notifications, CI/CD pipeline triggers.

---

## **Section 2: Error Handling and Timeouts**

### **Theory: Robust HTTP Error Handling**

Production systems must handle network failures gracefully:

**Common HTTP Errors:**
- **Connection errors**: Network unreachable, DNS failure
- **Timeout errors**: Server takes too long to respond
- **HTTP errors**: 4xx client errors, 5xx server errors
- **Parsing errors**: Invalid JSON, corrupt data

**Timeout Types:**
- **Connection timeout**: Time to establish connection
- **Read timeout**: Time waiting for server response
- **Total timeout**: Overall request time limit

**Retry Strategies:**
- **Exponential backoff**: 1s, 2s, 4s, 8s delays
- **Jitter**: Add randomness to prevent thundering herd
- **Max retries**: Limit total attempts
- **Idempotency**: Only retry safe operations (GET, PUT, DELETE)

### **Example 2.1: Error Handling and Timeouts**

In [2]:
import requests
from requests.exceptions import (
    ConnectionError, Timeout, HTTPError, 
    RequestException, TooManyRedirects
)
import time
import random

# Example 1: Timeout handling
print("Example 1: Timeout Handling\n")

try:
    # Test with a delayed response endpoint
    # timeout=(connect_timeout, read_timeout)
    response = requests.get(
        'https://httpbin.org/delay/10',
        timeout=(3, 5)  # 3s to connect, 5s to read
    )
    print(f"✓ Request succeeded: {response.status_code}")
except Timeout as e:
    print(f"✗ Request timed out: {e}")
except ConnectionError as e:
    print(f"✗ Connection failed: {e}")

# Example 2: HTTP error handling
print("\n" + "="*60)
print("Example 2: HTTP Error Handling\n")

urls_to_test = [
    ('https://httpbin.org/status/200', 'Success'),
    ('https://httpbin.org/status/404', 'Not Found'),
    ('https://httpbin.org/status/500', 'Server Error'),
]

for url, description in urls_to_test:
    try:
        response = requests.get(url, timeout=5)
        
        # Raise exception for 4xx and 5xx status codes
        response.raise_for_status()
        
        print(f"✓ {description}: {response.status_code}")
        
    except HTTPError as e:
        print(f"✗ {description}: HTTP {e.response.status_code} - {e}")
    except RequestException as e:
        print(f"✗ {description}: Request failed - {e}")

# Example 3: Comprehensive error handling function
print("\n" + "="*60)
print("Example 3: Production-Ready Request Function\n")

def safe_request(url: str, method: str = 'GET', max_retries: int = 3, 
                 timeout: int = 10, **kwargs) -> Optional[Dict[str, Any]]:
    """
    Make HTTP request with comprehensive error handling and retry logic.
    
    Args:
        url: Target URL
        method: HTTP method (GET, POST, etc.)
        max_retries: Maximum number of retry attempts
        timeout: Request timeout in seconds
        **kwargs: Additional arguments for requests library
    
    Returns:
        Response data as dict or None if failed
    """
    for attempt in range(max_retries):
        try:
            # Make request
            response = requests.request(
                method=method,
                url=url,
                timeout=timeout,
                **kwargs
            )
            
            # Check for HTTP errors
            response.raise_for_status()
            
            # Success
            print(f"  ✓ Attempt {attempt + 1}: Success ({response.status_code})")
            
            # Try to parse JSON, fallback to text
            try:
                return response.json()
            except ValueError:
                return {'text': response.text, 'status_code': response.status_code}
        
        except Timeout:
            print(f"  ✗ Attempt {attempt + 1}: Timeout after {timeout}s")
            if attempt < max_retries - 1:
                # Exponential backoff with jitter
                delay = (2 ** attempt) + random.uniform(0, 1)
                print(f"    Retrying in {delay:.1f}s...")
                time.sleep(delay)
        
        except HTTPError as e:
            status_code = e.response.status_code
            print(f"  ✗ Attempt {attempt + 1}: HTTP {status_code}")
            
            # Don't retry client errors (4xx) except 429 (rate limit)
            if 400 <= status_code < 500 and status_code != 429:
                print(f"    Client error - not retrying")
                return None
            
            if attempt < max_retries - 1:
                delay = (2 ** attempt) + random.uniform(0, 1)
                print(f"    Retrying in {delay:.1f}s...")
                time.sleep(delay)
        
        except ConnectionError as e:
            print(f"  ✗ Attempt {attempt + 1}: Connection error")
            if attempt < max_retries - 1:
                delay = (2 ** attempt) + random.uniform(0, 1)
                print(f"    Retrying in {delay:.1f}s...")
                time.sleep(delay)
        
        except TooManyRedirects:
            print(f"  ✗ Attempt {attempt + 1}: Too many redirects")
            return None
        
        except RequestException as e:
            print(f"  ✗ Attempt {attempt + 1}: Request failed - {e}")
            if attempt < max_retries - 1:
                delay = (2 ** attempt) + random.uniform(0, 1)
                print(f"    Retrying in {delay:.1f}s...")
                time.sleep(delay)
    
    print(f"  ✗ All {max_retries} attempts failed")
    return None

# Test the safe_request function
test_cases = [
    ('https://httpbin.org/get', 'Successful request'),
    ('https://httpbin.org/status/500', 'Server error with retry'),
    ('https://httpbin.org/status/404', 'Client error (no retry)'),
]

for url, description in test_cases:
    print(f"\nTesting: {description}")
    result = safe_request(url, max_retries=3, timeout=5)
    if result:
        print(f"  Final result: Got response data")
    else:
        print(f"  Final result: Request failed")

Example 1: Timeout Handling

✗ Request timed out: HTTPSConnectionPool(host='httpbin.org', port=443): Read timed out. (read timeout=5)

Example 2: HTTP Error Handling

✗ Request timed out: HTTPSConnectionPool(host='httpbin.org', port=443): Read timed out. (read timeout=5)

Example 2: HTTP Error Handling

✗ Success: HTTP 503 - 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/status/200
✗ Success: HTTP 503 - 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/status/200
✗ Not Found: HTTP 503 - 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/status/404
✗ Not Found: HTTP 503 - 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/status/404
✗ Server Error: HTTP 503 - 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/status/500

Example 3: Production-Ready Request Function


Testing: Successful request
✗ Server Error: HTTP 503 - 503 Server Error: Servi

### **Explanation:**

**Exception Hierarchy:**
- **`RequestException`**: Base class for all requests exceptions
- **`ConnectionError`**: Network-level connection failures
- **`Timeout`**: Request exceeded timeout limit
- **`HTTPError`**: HTTP response indicates error (4xx, 5xx)
- **`TooManyRedirects`**: Exceeded maximum redirect limit

**Timeout Configuration:**
- **Tuple format**: `timeout=(connect_timeout, read_timeout)`
- **Single value**: `timeout=5` applies to both connect and read
- **Connection timeout**: Time to establish TCP connection
- **Read timeout**: Time server has to send first byte of response

**`raise_for_status()` Method:**
- Raises `HTTPError` for status codes >= 400
- Does nothing for 2xx and 3xx codes
- Essential for catching HTTP-level errors

**Retry Logic Best Practices:**
- **Don't retry 4xx errors** (except 429 Rate Limit)
- **Always retry 5xx errors** (server issues are often transient)
- **Retry connection errors** (network issues often resolve quickly)
- **Use exponential backoff** to avoid overwhelming servers
- **Add jitter** (randomness) to prevent synchronized retry storms

**Exponential Backoff Calculation:**
```python
delay = (2 ** attempt) + random.uniform(0, 1)
# Attempt 0: 1 + [0-1] = 1-2 seconds
# Attempt 1: 2 + [0-1] = 2-3 seconds  
# Attempt 2: 4 + [0-1] = 4-5 seconds
```

**DevOps Use Case:** API monitoring, webhook delivery, microservice communication, deployment health checks.

---

## **Section 3: Working with REST APIs - GitHub API**

### **Theory: REST API Integration**

REST APIs follow standard patterns for resource management:

**GitHub API Features:**
- **Public endpoints**: No authentication required for public data
- **Authenticated endpoints**: Require personal access token
- **Rate limiting**: 60 req/hour (unauthenticated), 5000 req/hour (authenticated)
- **Pagination**: Use `page` and `per_page` parameters
- **HATEOAS**: Links to related resources in response headers

**Authentication Methods:**
- **Personal Access Token**: `Authorization: token <TOKEN>`
- **OAuth Apps**: Full OAuth2 flow
- **GitHub Apps**: For organization-wide integrations

**Common GitHub API Endpoints:**
- `/users/{username}`: User information
- `/repos/{owner}/{repo}`: Repository details
- `/repos/{owner}/{repo}/issues`: Repository issues
- `/search/repositories`: Search repositories

### **Example 3.1: GitHub API Client**

In [4]:
import requests
import os
from typing import Dict, List, Optional, Any
from datetime import datetime

class GitHubAPIClient:
    """
    GitHub API client with error handling, rate limiting, and authentication.
    """
    
    def __init__(self, token: Optional[str] = None):
        """
        Initialize GitHub API client.
        
        Args:
            token: Personal access token (optional, increases rate limit)
        """
        self.base_url = "https://api.github.com"
        self.token = token or os.environ.get('GITHUB_TOKEN')
        self.session = requests.Session()
        
        # Set up headers
        self.session.headers.update({
            'Accept': 'application/vnd.github.v3+json',
            'User-Agent': 'Python-DevOps-Client/1.0'
        })
        
        if self.token:
            self.session.headers.update({
                'Authorization': f'token {self.token}'
            })
            print(" Authenticated with GitHub token")
        else:
            print("  No token provided - using unauthenticated access (60 req/hour)")
    
    def _request(self, method: str, endpoint: str, **kwargs) -> Optional[Dict[str, Any]]:
        """
        Make API request with error handling.
        """
        url = f"{self.base_url}{endpoint}"
        
        try:
            response = self.session.request(method, url, timeout=10, **kwargs)
            
            # Check rate limit
            self._check_rate_limit(response.headers)
            
            response.raise_for_status()
            return response.json()
            
        except HTTPError as e:
            if e.response.status_code == 404:
                print(f"✗ Not found: {endpoint}")
            elif e.response.status_code == 403:
                print(f"✗ Forbidden: {e.response.json().get('message', 'Access denied')}")
            else:
                print(f"✗ HTTP Error {e.response.status_code}: {e}")
            return None
            
        except RequestException as e:
            print(f"✗ Request failed: {e}")
            return None
    
    def _check_rate_limit(self, headers: Dict[str, str]):
        """Check and display rate limit information."""
        if 'X-RateLimit-Remaining' in headers:
            remaining = int(headers['X-RateLimit-Remaining'])
            limit = int(headers['X-RateLimit-Limit'])
            
            if remaining < 10:
                reset_time = datetime.fromtimestamp(int(headers['X-RateLimit-Reset']))
                print(f"  Rate limit low: {remaining}/{limit} - Resets at {reset_time}")
    
    def get_user(self, username: str) -> Optional[Dict[str, Any]]:
        """Get user information."""
        return self._request('GET', f'/users/{username}')
    
    def get_repo(self, owner: str, repo: str) -> Optional[Dict[str, Any]]:
        """Get repository information."""
        return self._request('GET', f'/repos/{owner}/{repo}')
    
    def list_user_repos(self, username: str, per_page: int = 10) -> List[Dict[str, Any]]:
        """List user's public repositories."""
        result = self._request('GET', f'/users/{username}/repos', 
                               params={'per_page': per_page, 'sort': 'updated'})
        return result if result else []
    
    def search_repositories(self, query: str, per_page: int = 5) -> List[Dict[str, Any]]:
        """Search repositories."""
        result = self._request('GET', '/search/repositories',
                               params={'q': query, 'per_page': per_page, 'sort': 'stars'})
        return result.get('items', []) if result else []
    
    def get_repo_languages(self, owner: str, repo: str) -> Optional[Dict[str, int]]:
        """Get programming languages used in repository."""
        return self._request('GET', f'/repos/{owner}/{repo}/languages')
    
    def get_rate_limit(self) -> Optional[Dict[str, Any]]:
        """Get current rate limit status."""
        return self._request('GET', '/rate_limit')


# ===================== DEMONSTRATION =====================

print("=== GITHUB API CLIENT DEMONSTRATION ===\n")

# Initialize client (will use token from environment if available)
client = GitHubAPIClient()

# Example 1: Get user information
print("\n" + "="*60)
print(" Example 1: Get User Information\n")

user_data = client.get_user('torvalds')
if user_data:
    print(f"Name: {user_data.get('name')}")
    print(f"Username: {user_data['login']}")
    print(f"Public Repos: {user_data['public_repos']}")
    print(f"Followers: {user_data['followers']}")
    print(f"Created: {user_data['created_at']}")
    print(f"Bio: {user_data.get('bio', 'N/A')}")

# Example 2: Get repository information
print("\n" + "="*60)
print(" Example 2: Get Repository Details\n")

repo_data = client.get_repo('python', 'cpython')
if repo_data:
    print(f"Name: {repo_data['full_name']}")
    print(f"Description: {repo_data.get('description', 'N/A')}")
    print(f"Stars:  {repo_data['stargazers_count']:,}")
    print(f"Forks:  {repo_data['forks_count']:,}")
    print(f"Open Issues: {repo_data['open_issues_count']}")
    print(f"Language: {repo_data.get('language', 'N/A')}")
    print(f"Last Updated: {repo_data['updated_at']}")

# Example 3: List user repositories
print("\n" + "="*60)
print(" Example 3: List User Repositories\n")

repos = client.list_user_repos('github', per_page=5)
if repos:
    print(f"Found {len(repos)} repositories:\n")
    for repo in repos:
        print(f"  • {repo['name']}")
        print(f"     {repo['stargazers_count']} stars | "
              f"Language: {repo.get('language', 'N/A')}")
        print(f"    {repo.get('description', 'No description')[:60]}...")
        print()

# Example 4: Search repositories
print("\n" + "="*60)
print(" Example 4: Search Repositories\n")

results = client.search_repositories('devops python', per_page=3)
if results:
    print(f"Top {len(results)} results for 'devops python':\n")
    for repo in results:
        print(f"   {repo['full_name']}")
        print(f"     {repo['stargazers_count']:,} stars")
        print(f"    {repo.get('description', 'No description')[:60]}...")
        print()

# Example 5: Get repository languages
print("\n" + "="*60)
print(" Example 5: Repository Languages\n")

languages = client.get_repo_languages('microsoft', 'vscode')
if languages:
    total_bytes = sum(languages.values())
    print("Language breakdown:")
    for lang, bytes_count in sorted(languages.items(), key=lambda x: x[1], reverse=True)[:5]:
        percentage = (bytes_count / total_bytes) * 100
        print(f"  {lang}: {percentage:.1f}%")

# Example 6: Check rate limit
print("\n" + "="*60)
print(" Example 6: Rate Limit Status\n")

rate_limit = client.get_rate_limit()
if rate_limit:
    core = rate_limit['resources']['core']
    print(f"Requests remaining: {core['remaining']}/{core['limit']}")
    reset_time = datetime.fromtimestamp(core['reset'])
    print(f"Resets at: {reset_time.strftime('%Y-%m-%d %H:%M:%S')}")

=== GITHUB API CLIENT DEMONSTRATION ===

  No token provided - using unauthenticated access (60 req/hour)

 Example 1: Get User Information

Name: Linus Torvalds
Username: torvalds
Public Repos: 9
Followers: 265458
Created: 2011-09-03T15:26:22Z
Bio: None

 Example 2: Get Repository Details

Name: Linus Torvalds
Username: torvalds
Public Repos: 9
Followers: 265458
Created: 2011-09-03T15:26:22Z
Bio: None

 Example 2: Get Repository Details

Name: python/cpython
Description: The Python programming language
Stars:  70,287
Forks:  33,660
Open Issues: 9255
Language: Python
Last Updated: 2025-12-13T16:39:38Z

 Example 3: List User Repositories

Name: python/cpython
Description: The Python programming language
Stars:  70,287
Forks:  33,660
Open Issues: 9255
Language: Python
Last Updated: 2025-12-13T16:39:38Z

 Example 3: List User Repositories

Found 5 repositories:

  • spec-kit
     55218 stars | Language: Python
    💫 Toolkit to help you get started with Spec-Driven Developme...

  • copilo

### **Explanation:**

**Session Object Benefits:**
- **`requests.Session()`**: Reuses TCP connections (connection pooling)
- **Persistent headers**: Set once, applied to all requests
- **Cookie persistence**: Maintains cookies across requests
- **Better performance**: Avoids connection overhead for multiple requests

**Authentication:**
- **Personal Access Token**: `Authorization: token <TOKEN>`
- **Environment variable**: `os.environ.get('GITHUB_TOKEN')` for security
- Never hardcode tokens in source code
- Token increases rate limit from 60 to 5000 requests/hour

**Rate Limit Handling:**
- **Headers**: `X-RateLimit-Remaining`, `X-RateLimit-Limit`, `X-RateLimit-Reset`
- **Proactive checking**: Warn when limit is low
- **Reset time**: Unix timestamp when limit resets
- **Best practice**: Implement backoff when limit is reached

**API Response Patterns:**
- **User endpoint**: Returns user profile data
- **Repository endpoint**: Returns repo metadata
- **Search endpoint**: Returns `items` array with results
- **Pagination**: Use `per_page` and `page` parameters

**Error Handling:**
- **404 Not Found**: Resource doesn't exist
- **403 Forbidden**: Rate limit or permission issue
- **401 Unauthorized**: Invalid or missing authentication
- **422 Unprocessable**: Validation error in request data

**DevOps Use Case:** Repository automation, CI/CD integration, code review automation, release management, issue tracking.

---

## **Section 4: SSH Automation with Paramiko**

### **Theory: SSH and Remote Server Management**

SSH (Secure Shell) is essential for remote server management in DevOps:

**SSH Use Cases:**
- **Remote command execution**: Run commands on remote servers
- **File transfer**: SFTP for secure file uploads/downloads
- **Tunneling**: Create secure tunnels for port forwarding
- **Configuration management**: Update configs on remote systems

**Paramiko Features:**
- **SSHClient**: High-level SSH connection management
- **SFTPClient**: File transfer operations
- **Key-based authentication**: More secure than passwords
- **Command execution**: Run commands and capture output

**Security Best Practices:**
- **Use SSH keys** instead of passwords
- **Disable password authentication** on servers
- **Restrict key permissions**: `chmod 600 ~/.ssh/id_rsa`
- **Use ssh-agent** for key management
- **Rotate keys regularly**

### **Example 4.1: SSH Client with Paramiko**

In [1]:
import paramiko
from typing import Optional, Tuple, List
import os
from pathlib import Path
import io

class SSHManager:
    """
    SSH connection manager with command execution and file transfer.
    
    Note: This is a demonstration. In production, you would connect to actual servers.
    """
    
    def __init__(self, hostname: str, username: str, 
                 password: Optional[str] = None,
                 key_filename: Optional[str] = None,
                 port: int = 22):
        """
        Initialize SSH manager.
        
        Args:
            hostname: Server hostname or IP
            username: SSH username
            password: Password (optional if using key)
            key_filename: Path to private key file
            port: SSH port (default: 22)
        """
        self.hostname = hostname
        self.username = username
        self.password = password
        self.key_filename = key_filename
        self.port = port
        self.client = None
        self.sftp = None
    
    def connect(self) -> bool:
        """
        Establish SSH connection.
        
        Returns:
            True if connected successfully
        """
        try:
            self.client = paramiko.SSHClient()
            
            # Auto-accept unknown host keys (use with caution in production!)
            self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            
            connect_kwargs = {
                'hostname': self.hostname,
                'username': self.username,
                'port': self.port,
                'timeout': 10
            }
            
            # Use key-based auth if key provided, otherwise password
            if self.key_filename:
                connect_kwargs['key_filename'] = self.key_filename
                print(f"Connecting with SSH key: {self.key_filename}")
            elif self.password:
                connect_kwargs['password'] = self.password
                print(f"Connecting with password")
            else:
                print("No authentication method provided")
                return False
            
            self.client.connect(**connect_kwargs)
            print(f"✓ Connected to {self.hostname}")
            return True
            
        except paramiko.AuthenticationException:
            print(f"✗ Authentication failed for {self.username}@{self.hostname}")
            return False
        except paramiko.SSHException as e:
            print(f"✗ SSH error: {e}")
            return False
        except Exception as e:
            print(f"✗ Connection error: {e}")
            return False
    
    def execute_command(self, command: str, timeout: int = 30) -> Tuple[int, str, str]:
        """
        Execute command on remote server.
        
        Args:
            command: Command to execute
            timeout: Command timeout in seconds
        
        Returns:
            Tuple of (exit_code, stdout, stderr)
        """
        if not self.client:
            print("✗ Not connected. Call connect() first.")
            return (-1, "", "Not connected")
        
        try:
            print(f"Executing: {command}")
            
            stdin, stdout, stderr = self.client.exec_command(command, timeout=timeout)
            
            # Wait for command to complete
            exit_code = stdout.channel.recv_exit_status()
            
            stdout_str = stdout.read().decode('utf-8')
            stderr_str = stderr.read().decode('utf-8')
            
            if exit_code == 0:
                print(f"✓ Command succeeded (exit code: {exit_code})")
            else:
                print(f"✗ Command failed (exit code: {exit_code})")
            
            return (exit_code, stdout_str, stderr_str)
            
        except paramiko.SSHException as e:
            print(f"✗ SSH error: {e}")
            return (-1, "", str(e))
        except Exception as e:
            print(f"✗ Execution error: {e}")
            return (-1, "", str(e))
    
    def execute_commands(self, commands: List[str]) -> List[Tuple[str, int, str, str]]:
        """
        Execute multiple commands sequentially.
        
        Returns:
            List of (command, exit_code, stdout, stderr) tuples
        """
        results = []
        
        for command in commands:
            exit_code, stdout, stderr = self.execute_command(command)
            results.append((command, exit_code, stdout, stderr))
            
            # Stop on first failure if desired
            # if exit_code != 0:
            #     break
        
        return results
    
    def upload_file(self, local_path: str, remote_path: str) -> bool:
        """
        Upload file to remote server via SFTP.
        
        Args:
            local_path: Local file path
            remote_path: Remote destination path
        
        Returns:
            True if successful
        """
        if not self.client:
            print("✗ Not connected")
            return False
        
        try:
            if not self.sftp:
                self.sftp = self.client.open_sftp()
            
            print(f"Uploading {local_path} -> {remote_path}")
            self.sftp.put(local_path, remote_path)
            print(f"✓ Upload successful")
            return True
            
        except FileNotFoundError:
            print(f"✗ Local file not found: {local_path}")
            return False
        except Exception as e:
            print(f"✗ Upload failed: {e}")
            return False
    
    def download_file(self, remote_path: str, local_path: str) -> bool:
        """
        Download file from remote server via SFTP.
        
        Args:
            remote_path: Remote file path
            local_path: Local destination path
        
        Returns:
            True if successful
        """
        if not self.client:
            print("✗ Not connected")
            return False
        
        try:
            if not self.sftp:
                self.sftp = self.client.open_sftp()
            
            print(f"Downloading {remote_path} -> {local_path}")
            self.sftp.get(remote_path, local_path)
            print(f"✓ Download successful")
            return True
            
        except FileNotFoundError:
            print(f"✗ Remote file not found: {remote_path}")
            return False
        except Exception as e:
            print(f"✗ Download failed: {e}")
            return False
    
    def list_directory(self, remote_path: str = '.') -> Optional[List[str]]:
        """
        List contents of remote directory.
        
        Args:
            remote_path: Remote directory path
        
        Returns:
            List of filenames or None if failed
        """
        if not self.client:
            print("✗ Not connected")
            return None
        
        try:
            if not self.sftp:
                self.sftp = self.client.open_sftp()
            
            files = self.sftp.listdir(remote_path)
            return files
            
        except Exception as e:
            print(f"✗ List directory failed: {e}")
            return None
    
    def close(self):
        """Close SSH and SFTP connections."""
        if self.sftp:
            self.sftp.close()
            self.sftp = None
        
        if self.client:
            self.client.close()
            self.client = None
            print(f"✓ Disconnected from {self.hostname}")
    
    def __enter__(self):
        """Context manager entry."""
        self.connect()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit."""
        self.close()


# ===================== DEMONSTRATION =====================

print("=== SSH MANAGER DEMONSTRATION ===\n")
print(" Note: This example demonstrates the API.")
print("To use with real servers, provide valid credentials.\n")

# Example usage (commented out - requires actual server)
"""
# Example 1: Connect and execute commands
with SSHManager(
    hostname='your-server.com',
    username='devops',
    key_filename='/home/user/.ssh/id_rsa'
) as ssh:
    
    # Execute single command
    exit_code, stdout, stderr = ssh.execute_command('uname -a')
    print(f"Output: {stdout}")
    
    # Execute multiple commands
    commands = [
        'df -h',                    # Disk usage
        'free -m',                  # Memory usage
        'uptime',                   # System uptime
        'docker ps',                # Running containers
        'systemctl status nginx'    # Service status
    ]
    
    results = ssh.execute_commands(commands)
    for cmd, exit_code, stdout, stderr in results:
        print(f"\nCommand: {cmd}")
        print(f"Exit Code: {exit_code}")
        print(f"Output: {stdout[:200]}")
"""

# Example 2: Demonstrate SSH connection patterns
print("="*60)
print("SSH Connection Patterns\n")

print("Pattern 1: Key-based authentication (recommended)")
print("```python")
print("ssh = SSHManager(")
print("    hostname='server.example.com',")
print("    username='deploy',")
print("    key_filename='~/.ssh/id_rsa'")
print(")")
print("```\n")

print("Pattern 2: Password authentication (less secure)")
print("```python")
print("ssh = SSHManager(")
print("    hostname='server.example.com',")
print("    username='admin',")
print("    password=os.environ.get('SSH_PASSWORD')")
print(")")
print("```\n")

print("Pattern 3: Custom port")
print("```python")
print("ssh = SSHManager(")
print("    hostname='server.example.com',")
print("    username='devops',")
print("    key_filename='~/.ssh/id_rsa',")
print("    port=2222")
print(")")
print("```\n")

# Example 3: Common DevOps commands
print("="*60)
print(" Common DevOps SSH Commands\n")

devops_commands = {
    'System Info': [
        'uname -a',              # Kernel version
        'cat /etc/os-release',   # OS information
        'uptime',                # System uptime
    ],
    'Resource Monitoring': [
        'top -bn1 | head -20',   # CPU/memory snapshot
        'df -h',                 # Disk usage
        'free -m',               # Memory usage
        'iostat',                # I/O statistics
    ],
    'Docker Operations': [
        'docker ps',             # Running containers
        'docker images',         # Local images
        'docker stats --no-stream',  # Container resources
    ],
    'Service Management': [
        'systemctl status nginx',     # Check service
        'journalctl -u nginx -n 50',  # Service logs
        'systemctl list-units --failed',  # Failed services
    ],
    'Log Analysis': [
        'tail -n 100 /var/log/syslog',  # System logs
        'tail -n 50 /var/log/nginx/error.log',  # Web server errors
        'grep ERROR /var/log/application.log',  # Application errors
    ]
}

for category, commands in devops_commands.items():
    print(f"\n{category}:")
    for cmd in commands:
        print(f"  • {cmd}")

print("\n" + "="*60)
print("SSH Manager ready for production use")
print("Provide real server credentials to test operations")

=== SSH MANAGER DEMONSTRATION ===

 Note: This example demonstrates the API.
To use with real servers, provide valid credentials.

SSH Connection Patterns

Pattern 1: Key-based authentication (recommended)
```python
ssh = SSHManager(
    hostname='server.example.com',
    username='deploy',
    key_filename='~/.ssh/id_rsa'
)
```

Pattern 2: Password authentication (less secure)
```python
ssh = SSHManager(
    hostname='server.example.com',
    username='admin',
    password=os.environ.get('SSH_PASSWORD')
)
```

Pattern 3: Custom port
```python
ssh = SSHManager(
    hostname='server.example.com',
    username='devops',
    key_filename='~/.ssh/id_rsa',
    port=2222
)
```

 Common DevOps SSH Commands


System Info:
  • uname -a
  • cat /etc/os-release
  • uptime

Resource Monitoring:
  • top -bn1 | head -20
  • df -h
  • free -m
  • iostat

Docker Operations:
  • docker ps
  • docker images
  • docker stats --no-stream

Service Management:
  • systemctl status nginx
  • journalctl -u ng

**Explanation:**

The `SSHManager` class provides a production-ready interface for SSH operations:

**Connection Management:**
- **Key-based authentication**: More secure than passwords, uses SSH key pairs
- **Password authentication**: Fallback option, credentials should be from environment variables
- **Auto-policy**: `AutoAddPolicy()` auto-accepts host keys (use `RejectPolicy()` in production for better security)
- **Context manager**: `with` statement ensures connections are properly closed

**Command Execution:**
- **Single commands**: `execute_command()` runs one command and captures output
- **Multiple commands**: `execute_commands()` runs a list sequentially
- **Exit codes**: Return codes indicate success (0) or failure (non-zero)
- **Timeouts**: Prevents hanging on long-running commands

**File Transfer (SFTP):**
- **Upload**: `upload_file()` transfers local files to remote server
- **Download**: `download_file()` retrieves files from remote server
- **Directory listing**: `list_directory()` shows remote directory contents
- **Binary safe**: Handles text and binary files correctly

**DevOps Use Cases:**
- **Deployment**: Upload application code, restart services
- **Monitoring**: Collect system metrics, check service status
- **Log collection**: Download logs for analysis
- **Batch operations**: Execute commands across multiple servers
- **Configuration management**: Update config files, manage services

**Security Best Practices:**
- Always use key-based authentication in production
- Store credentials in environment variables or secrets management systems
- Use `known_hosts` verification instead of `AutoAddPolicy()` for production
- Implement connection pooling for high-frequency operations
- Log all SSH operations for audit purposes

---

## **5. Advanced Topics & Best Practices**

### **5.1 Real-World DevOps Scenarios**

Here are practical examples of how networking and APIs are used in DevOps workflows:

#### **Scenario 1: Automated Deployment Pipeline**
```python
# 1. Download artifacts from GitHub release
github = GitHubAPIClient(token)
release = github.get_repo_details('myorg/myapp')

# 2. SSH to production server
with SSHManager(hostname='prod-01', username='deploy', key_filename='~/.ssh/deploy_key') as ssh:
    # Stop service
    ssh.execute_command('sudo systemctl stop myapp')
    
    # Upload new version
    ssh.upload_file('/tmp/myapp-v2.0.tar.gz', '/opt/myapp/myapp-v2.0.tar.gz')
    
    # Extract and restart
    ssh.execute_command('cd /opt/myapp && tar xzf myapp-v2.0.tar.gz')
    ssh.execute_command('sudo systemctl start myapp')
    
    # Verify service is running
    exit_code, stdout, stderr = ssh.execute_command('curl -f http://localhost:8080/health')
    if exit_code == 0:
        print("✓ Deployment successful")
```

#### **Scenario 2: Multi-Server Health Check**
```python
servers = [
    'web-01.example.com',
    'web-02.example.com',
    'web-03.example.com'
]

health_status = {}

for server in servers:
    try:
        response = requests.get(f'https://{server}/health', timeout=5)
        health_status[server] = {
            'status': 'healthy' if response.status_code == 200 else 'unhealthy',
            'response_time': response.elapsed.total_seconds(),
            'version': response.json().get('version', 'unknown')
        }
    except requests.RequestException as e:
        health_status[server] = {'status': 'down', 'error': str(e)}

# Send report to monitoring API
monitoring_api = 'https://monitoring.example.com/api/v1/health-report'
requests.post(monitoring_api, json=health_status)
```

#### **Scenario 3: Log Aggregation**
```python
servers = ['app-01', 'app-02', 'app-03']
log_path = '/var/log/application/error.log'

for server in servers:
    with SSHManager(hostname=server, username='devops', key_filename='~/.ssh/id_rsa') as ssh:
        # Download last 1000 lines of error logs
        exit_code, stdout, stderr = ssh.execute_command(f'tail -n 1000 {log_path}')
        
        if exit_code == 0:
            # Parse and send to logging service
            errors = [line for line in stdout.split('\n') if 'ERROR' in line]
            
            # Send to centralized logging (e.g., ELK, Splunk)
            logging_api = 'https://logs.example.com/api/v1/ingest'
            requests.post(logging_api, json={
                'server': server,
                'timestamp': datetime.now().isoformat(),
                'errors': errors
            })
```

---

### **5.2 Security Best Practices**

When working with APIs and SSH in production environments, follow these security guidelines:

#### **API Security**
1. **Never hardcode credentials**
   ```python
   # ❌ BAD - credentials in code
   token = 'ghp_abc123xyz'
   
   # ✅ GOOD - use environment variables
   token = os.environ.get('GITHUB_TOKEN')
   
   # ✅ BETTER - use secrets management
   from azure.keyvault.secrets import SecretClient
   token = secret_client.get_secret('github-token').value
   ```

2. **Use HTTPS only**
   ```python
   # ❌ BAD - HTTP is not encrypted
   response = requests.get('http://api.example.com/data')
   
   # ✅ GOOD - HTTPS encrypts traffic
   response = requests.get('https://api.example.com/data')
   ```

3. **Validate SSL certificates**
   ```python
   # ❌ BAD - disables SSL verification (security risk!)
   response = requests.get(url, verify=False)
   
   # ✅ GOOD - verify SSL certificate
   response = requests.get(url, verify=True)
   
   # ✅ CUSTOM - use custom CA bundle
   response = requests.get(url, verify='/path/to/ca-bundle.crt')
   ```

4. **Implement rate limiting**
   ```python
   from time import time, sleep
   
   class RateLimiter:
       def __init__(self, requests_per_second=10):
           self.requests_per_second = requests_per_second
           self.last_request = 0
       
       def wait_if_needed(self):
           elapsed = time() - self.last_request
           wait_time = (1.0 / self.requests_per_second) - elapsed
           if wait_time > 0:
               sleep(wait_time)
           self.last_request = time()
   ```

#### **SSH Security**
1. **Use key-based authentication**
   ```bash
   # Generate SSH key pair
   ssh-keygen -t rsa -b 4096 -C "deploy@example.com"
   
   # Copy public key to server
   ssh-copy-id -i ~/.ssh/id_rsa.pub user@server
   ```

2. **Set proper key permissions**
   ```bash
   chmod 700 ~/.ssh
   chmod 600 ~/.ssh/id_rsa
   chmod 644 ~/.ssh/id_rsa.pub
   chmod 644 ~/.ssh/known_hosts
   ```

3. **Verify host keys**
   ```python
   # ❌ BAD - auto-accepts any host (MITM risk!)
   client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
   
   # ✅ GOOD - use known_hosts file
   client.load_system_host_keys()
   client.set_missing_host_key_policy(paramiko.RejectPolicy())
   ```

4. **Use SSH agent forwarding cautiously**
   ```python
   # Only enable when necessary
   ssh.connect(hostname, username, allow_agent=False)
   ```

#### **General Best Practices**
- **Log all API and SSH operations** for audit trails
- **Implement timeouts** to prevent hanging operations
- **Use least privilege principle** - minimal necessary permissions
- **Rotate credentials regularly** (keys, tokens, passwords)
- **Monitor for suspicious activity** (failed logins, unusual API calls)
- **Encrypt sensitive data at rest** (local config files, cache)
- **Use connection pooling** for high-frequency operations
- **Implement circuit breakers** for external service failures

---

### **5.3 Performance Optimization**

#### **Connection Pooling**
Reuse connections instead of creating new ones for each request:

```python
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Create session with connection pooling
session = requests.Session()

# Configure retries
retry_strategy = Retry(
    total=3,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504]
)

adapter = HTTPAdapter(
    max_retries=retry_strategy,
    pool_connections=10,  # Number of connection pools
    pool_maxsize=20       # Max connections per pool
)

session.mount('https://', adapter)
session.mount('http://', adapter)

# Use session for all requests
for i in range(100):
    response = session.get('https://api.example.com/data')
```

#### **Async/Concurrent Requests**
Use `concurrent.futures` or `asyncio` for parallel operations:

```python
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_url(url):
    try:
        response = requests.get(url, timeout=5)
        return {'url': url, 'status': response.status_code, 'size': len(response.content)}
    except Exception as e:
        return {'url': url, 'error': str(e)}

urls = [f'https://api.example.com/data/{i}' for i in range(50)]

# Parallel execution with thread pool
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_url, url) for url in urls]
    
    results = []
    for future in as_completed(futures):
        results.append(future.result())

print(f"Fetched {len(results)} URLs")
```

#### **Caching**
Reduce redundant API calls with caching:

```python
from functools import lru_cache
from datetime import datetime, timedelta

class CachedAPIClient:
    def __init__(self):
        self.cache = {}
        self.cache_ttl = 300  # 5 minutes
    
    def get_with_cache(self, url):
        now = datetime.now()
        
        # Check cache
        if url in self.cache:
            cached_data, timestamp = self.cache[url]
            if now - timestamp < timedelta(seconds=self.cache_ttl):
                print(f"✓ Cache hit: {url}")
                return cached_data
        
        # Cache miss - fetch from API
        print(f"⟳ Cache miss: {url}")
        response = requests.get(url)
        data = response.json()
        
        # Store in cache
        self.cache[url] = (data, now)
        return data
```

---

### **5.4 Error Handling Patterns**

#### **Circuit Breaker Pattern**
Prevent cascading failures when external services are down:

```python
from datetime import datetime, timedelta
from enum import Enum

class CircuitState(Enum):
    CLOSED = "closed"      # Normal operation
    OPEN = "open"          # Failures detected, blocking calls
    HALF_OPEN = "half_open"  # Testing if service recovered

class CircuitBreaker:
    def __init__(self, failure_threshold=5, timeout=60):
        self.failure_threshold = failure_threshold
        self.timeout = timeout
        self.failure_count = 0
        self.last_failure_time = None
        self.state = CircuitState.CLOSED
    
    def call(self, func, *args, **kwargs):
        if self.state == CircuitState.OPEN:
            if datetime.now() - self.last_failure_time > timedelta(seconds=self.timeout):
                self.state = CircuitState.HALF_OPEN
                print("Circuit breaker: HALF_OPEN (testing)")
            else:
                raise Exception("Circuit breaker is OPEN - service unavailable")
        
        try:
            result = func(*args, **kwargs)
            
            # Success - reset counter
            if self.state == CircuitState.HALF_OPEN:
                print("Circuit breaker: CLOSED (service recovered)")
                self.state = CircuitState.CLOSED
            self.failure_count = 0
            return result
            
        except Exception as e:
            self.failure_count += 1
            self.last_failure_time = datetime.now()
            
            if self.failure_count >= self.failure_threshold:
                self.state = CircuitState.OPEN
                print(f"Circuit breaker: OPEN (failures: {self.failure_count})")
            
            raise e

# Usage
breaker = CircuitBreaker(failure_threshold=3, timeout=60)

def unreliable_api_call():
    response = requests.get('https://api.example.com/data', timeout=5)
    return response.json()

try:
    data = breaker.call(unreliable_api_call)
except Exception as e:
    print(f"Call failed: {e}")
```

#### **Retry with Exponential Backoff**
Already shown in Section 2, but here's the pattern:

```python
def exponential_backoff(attempt):
    """Calculate wait time: 1s, 2s, 4s, 8s, 16s..."""
    return min(2 ** attempt, 32)  # Cap at 32 seconds
```

#### **Graceful Degradation**
Provide fallback behavior when services fail:

```python
def get_user_data(user_id):
    try:
        # Try primary API
        response = requests.get(f'https://api.example.com/users/{user_id}', timeout=5)
        return response.json()
    except requests.RequestException:
        # Fallback to cache
        cached_data = get_from_cache(user_id)
        if cached_data:
            print("Using cached data (API unavailable)")
            return cached_data
        
        # Final fallback - default data
        print("Using default data (API and cache unavailable)")
        return {'id': user_id, 'name': 'Unknown', 'status': 'unavailable'}
```

---

## **6. Summary & Key Takeaways**

### **What We Learned**

In this session, we covered essential networking and API concepts for DevOps automation:

1. **HTTP Requests with `requests`**
   - Making GET, POST, and other HTTP requests
   - Handling query parameters, headers, and authentication
   - Working with JSON responses

2. **Error Handling & Timeouts**
   - Connection timeouts and read timeouts
   - Retry logic with exponential backoff
   - Handling different types of request exceptions

3. **REST APIs (GitHub API)**
   - Authentication with tokens
   - Making API calls (users, repos, search)
   - Rate limiting and pagination
   - Best practices for API clients

4. **SSH Automation with Paramiko**
   - Establishing SSH connections
   - Executing remote commands
   - File transfer with SFTP
   - Key-based vs password authentication

5. **Advanced Patterns**
   - Real-world DevOps scenarios (deployment, monitoring, log aggregation)
   - Security best practices (credentials, SSL, host keys)
   - Performance optimization (connection pooling, caching, async)
   - Error handling patterns (circuit breaker, graceful degradation)

### **Key Takeaways**

**Always handle errors** - Network operations can fail in many ways  
**Use timeouts** - Prevent hanging on unresponsive services  
**Implement retries** - Network issues are often transient  
**Secure credentials** - Never hardcode, use environment variables or secrets managers  
**Use HTTPS** - Encrypt all network traffic  
**Key-based SSH** - More secure than passwords for automation  
**Rate limiting** - Respect API limits and implement backoff  
**Connection pooling** - Reuse connections for better performance  
**Circuit breakers** - Prevent cascading failures  
**Logging & monitoring** - Track all API and SSH operations  

### **Next Steps**

- **Practice**: Try the examples with real APIs (GitHub, GitLab, Jenkins)
- **Build**: Create automation scripts for your deployment pipelines
- **Explore**: Learn about `asyncio` for async HTTP requests
- **Study**: Investigate other libraries like `httpx` (async), `fabric` (SSH wrapper)
- **Integrate**: Combine these skills with CI/CD tools (GitHub Actions, GitLab CI)

### **Additional Resources**

- **requests documentation**: https://requests.readthedocs.io/
- **Paramiko documentation**: https://www.paramiko.org/
- **GitHub API documentation**: https://docs.github.com/en/rest
- **REST API design best practices**: https://restfulapi.net/
- **Python asyncio**: https://docs.python.org/3/library/asyncio.html

---

## **7. Homework Exercises**

### **Exercise 1: API Integration**
Create a script that:
1. Fetches weather data from a free weather API (e.g., OpenWeatherMap)
2. Implements error handling and retries
3. Caches results for 10 minutes to reduce API calls
4. Logs all API requests to a file

### **Exercise 2: GitHub Repository Analyzer**
Build a tool that:
1. Takes a GitHub username as input
2. Lists all public repositories
3. For each repo, shows: stars, forks, language, last update
4. Sorts repositories by stars (most popular first)
5. Saves results to a JSON file

### **Exercise 3: SSH Server Monitor**
Create a monitoring script that:
1. Connects to multiple servers via SSH
2. Checks disk usage (`df -h`)
3. Checks memory usage (`free -m`)
4. Checks running processes (`ps aux | head -20`)
5. Sends alerts if disk usage > 80% or memory usage > 90%
6. Logs all checks with timestamps

### **Exercise 4: Automated Backup**
Implement a backup script that:
1. Connects to a remote server via SSH
2. Creates a compressed backup of `/var/log` directory
3. Downloads the backup file via SFTP
4. Verifies the backup file size
5. Deletes backups older than 7 days (both local and remote)

### **Bonus Challenge: Deployment Pipeline**
Build a mini deployment pipeline that:
1. Fetches latest release from GitHub API
2. Downloads release artifacts
3. Connects to production server via SSH
4. Stops the running service
5. Uploads new version
6. Starts the service
7. Verifies health endpoint returns 200
8. Rolls back if health check fails

---